# Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns 

### Contexto

No Airbnb, qualquer pessoa que tenha um quarto ou um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles (e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio, desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes extras, exigência de verificação de identidade do locador, etc.

### Nosso objetivo

Construir um modelo de previsão de preço que permita uma pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesmas características) ou não.

### O que temos disponível, inspirações e créditos

As bases de dados foram retiradas do site kaggle: https://www.kaggle.com/allanbruno/airbnb-rio-de-janeiro



### Importar Bibliotecas e Bases de Dados

In [1]:
import pandas as pd
import pathlib

### Consolidar Base de Dados

In [2]:
#Importamos uma base de dados mas temos que percorer todos as bases de dados
  #Usamos o pathlib
caminhos_bases =  pathlib.Path('dataset')

#Percorrer todos
for arquivo in caminhos_bases.iterdir():
    print(arquivo.name)

abril2018.csv
abril2019.csv
abril2020.csv
agosto2018.csv
agosto2019.csv
dezembro2018.csv
dezembro2019.csv
fevereiro2019.csv
fevereiro2020.csv
janeiro2019.csv
janeiro2020.csv
julho2018.csv
julho2019.csv
junho2019.csv
maio2018.csv
maio2019.csv
maio2020.csv
maro2019.csv
maro2020.csv
novembro2019.csv
novrmbro2018.csv
outubro2018.csv
outubro2019.csv
setembro2018.csv
setembro2019.csv


### Se tivermos muitas colunas, já vamos identificar quais colunas podemos excluir

### Tratar Valores Faltando

### Verificar Tipos de Dados em cada coluna

### Análise Exploratória e Tratar Outliers

### Encoding

### Modelo de Previsão

### Análise do Melhor Modelo

### Ajustes e Melhorias no Melhor Modelo